In [1]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import openai

In [3]:
#Se debe cargar un archivo .env con api de OpenAI
load_dotenv("../../../../../.env.txt")
api_key = os.getenv('OPENAI_API_KEY')

In [5]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

In [7]:
# se carga la base de datos
import pandas as pd
df = pd.read_csv("Recomendaciones_para_el_Estres.csv")
print(df.head())

   ID  Edad Género Estado Civil Tipo de Contrato Nivel de Responsabilidad  \
0  52    45      M       Casado   Tiempo Parcial                     Bajo   
1  93    37      M      Soltero  Tiempo Completo                     Bajo   
2  15    35      M   Divorciado   Tiempo Parcial                    Medio   
3  72    27      F   Divorciado   Tiempo Parcial                     Bajo   
4  61    26      M      Soltero  Tiempo Completo                    Medio   

   Satisfacción Laboral  Nivel de Estrés  \
0                     1                2   
1                     3               10   
2                     3                2   
3                     1                2   
4                     4                5   

                                               Texto             Fecha y Hora  \
0  Hoy fue un día muy satisfactorio, cumplí con t...  2024-09-06 15:06:38.986   
1  Hoy fue un día muy desorganizado, pero logré s...  2024-12-10 08:06:38.986   
2  El estrés no me deja disfr

In [9]:
system_prompt = (
    "Eres un asistente de recursos humanos que analiza la frase que te da un empleado "
    "y solo proporciona un numero para predecir el estado de estrés: nivel de estres entre 1 y 10. "
    "Responde en Markdown.")

In [11]:
# Se pide al usuario ingresar estado civil, edad, contrato laboral y una frase de como se siente
edad = input("Introduce tu edad")
estCiv = input("Introduce tu estado civil (Casado/Soltero/Divorciado/Viudo)")
genero = input("Introduce tu género (M/F)")
textoUsuario = input("En una frase describe cómo estuvo tu día laboral")

Introduce tu edad 30
Introduce tu estado civil (Casado/Soltero/Divorciado/Viudo) Casado
Introduce tu género (M/F) F
En una frase describe cómo estuvo tu día laboral Terrible, no me rindió


In [13]:
# Mensajes para el modelo
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": textoUsuario}
]

# Llamada al modelo
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0.5,
    max_tokens=150
)

# Respuesta
estressGPT = response.choices[0].message.content

# Mostrar respuesta
print(estressGPT)

7


In [15]:
rango_edad = 5 
rango_estress = 1
filtro = (
    (df['Género'] == genero) &
    (df['Estado Civil'] == estCiv) &
    (df['Edad'].between(int(edad) - rango_edad, int(edad) + rango_edad)) &
    (df['Nivel de Estrés'].between(int(estressGPT) - rango_estress, int(estressGPT) + rango_estress))    
)

personas_similares = df[filtro]

print("\nPersonas similares encontradas:")
print(personas_similares)


Personas similares encontradas:
       ID  Edad Género Estado Civil Tipo de Contrato Nivel de Responsabilidad  \
5      21    34      F       Casado  Tiempo Completo                     Alto   
69     36    30      F       Casado  Tiempo Completo                     Alto   
102    60    32      F       Casado   Tiempo Parcial                    Medio   
118     8    31      F       Casado   Tiempo Parcial                     Alto   
120    35    27      F       Casado   Tiempo Parcial                    Medio   
140    35    27      F       Casado   Tiempo Parcial                    Medio   
168    36    30      F       Casado  Tiempo Completo                     Alto   
205    44    30      F       Casado   Tiempo Parcial                     Alto   
362    35    27      F       Casado   Tiempo Parcial                    Medio   
418    60    32      F       Casado   Tiempo Parcial                    Medio   
527   100    31      F       Casado   Tiempo Parcial                    Medi

In [126]:
if not personas_similares.empty:
    recomendaciones = personas_similares['Consejo para el Estrés']

    print("\nTe recomendamos hacer lo siguiente: ")
    print(recomendaciones.sample(n=1))
else:
    print("\nNo se encontraron personas similares en la base de datos. Intenta con otros datos.")


Te recomendamos hacer lo siguiente: 
323    Tómate un momento para respirar profundamente ...
Name: Consejo para el Estrés, dtype: object
